In [47]:
from __future__ import annotations
import re
import json
from pathlib import Path

import numpy as np
import pandas as pd

import xlwings as xw
from pandas.core.dtypes.inference import is_integer


In [8]:
def check_is_file(*filepaths):
    files_not_found = []
    for filepath in filepaths:
        _filepath = Path(filepath)
        if not _filepath.is_file():
            files_not_found.append(filepath)
    if len(files_not_found) > 0:
        plural = "" if len(files_not_found) == 1 else "s"
        past_tense_form = "was" if len(files_not_found) == 1 else "were"
        exception_message = (
            f"The following file{plural} {past_tense_form}n't found: "
            + ", ".join(files_not_found)
        )
        raise FileNotFoundError(exception_message)


# Helper function to autofit column widths with a minimum width
def autofit_columns(ws):
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter  # Get the column name
        for cell in col:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        adjusted_width = max(max_length + 2, 10)  # Minimum width set to 10
        ws.column_dimensions[column].width = adjusted_width


# Convert input and output JSON files to dataframes
def explode_quality_rows(
    df: pd.DataFrame,
    quality_col_prefix: str = "quality_",
) -> pd.DataFrame:
    """
    Explode the `'quality_*'` columns into rows.

    Parameters
    ----------
    df : pd.DataFrame
        The dataframe with the output specifications with the quality columns
        that contain dictionaries of quality parameters to be extracted.
    quality_col_prefix : str, default='quality_'
        The prefix of the quality columns that contain the output pile specifications
        that need to be extracted into different columns.

    Returns
    -------
    pd.DataFrame
        The `pandas.DataFrame` with the quality dictionaries extracted into
        new columns.
    """
    # Create a new DataFrame to store exploded rows
    exploded_df = pd.DataFrame()

    for index, row in df.iterrows():
        quality_dict_list = [
            value for key, value in row.items() if str(key).startswith(quality_col_prefix)
        ]

        # Convert the list of quality dictionaries into a DataFrame
        quality_df = pd.DataFrame(quality_dict_list)

        # Repeat the original columns for each exploded row
        repeated_columns = pd.DataFrame(
            [
                row.drop(
                    labels=[
                        col for col in df.columns if col.startswith(quality_col_prefix)
                    ]
                )
            ]
            * len(quality_df)
        )

        # Concatenate the repeated columns with the quality columns
        exploded_row_df = pd.concat(
            [
                repeated_columns.reset_index(drop=True),
                quality_df.reset_index(drop=True),
            ],
            axis=1,
        )

        # Append to the exploded DataFrame
        exploded_df = pd.concat([exploded_df, exploded_row_df], ignore_index=True)

    return exploded_df


def assign_engines_to_stockpiles(
    stockpiles_df: pd.DataFrame, engines_df: pd.DataFrame
) -> pd.DataFrame:
    """
    Assign engines to stockpiles based on matching yards and rails.

    Parameters
    ----------
    stockpiles_df : pd.DataFrame
        A `pandas.DataFrame` containing stockpile information including
        'rails' and 'yard'.
    engines_df : pd.DataFrame
        A `pandas.DataFrame` containing engine information including 'yards' and 'rail'.

    Returns
    -------
    pd.DataFrame
        Updated `stockpiles_df` with an 'engines' column listing the assigned engine IDs.
    """
    stockpiles_df["engines"] = [[] for _ in range(stockpiles_df.shape[0])]

    for idx, stockpile in stockpiles_df.iterrows():
        assigned_engines = [
            eng_row["id"]
            for _, eng_row in engines_df.iterrows()
            if stockpile["yard"] in eng_row["yards"]
            and eng_row["rail"] in stockpile["rails"]
        ]
        stockpiles_df.at[idx, "engines"] = assigned_engines

    return stockpiles_df


def extract_quality_ini_values(
    stockpiles_df: pd.DataFrame, quality_prefix: str = "qualityIni"
) -> pd.DataFrame:
    """
    Extract initial quality values from nested dictionaries in the stockpiles DataFrame
    and add them as new columns.

    Parameters
    ----------
    stockpiles_df : pd.DataFrame
        A `pandas.DataFrame` containing stockpile information, including quality parameters.
    quality_prefix : str, default='qualityIni'
        Prefix used in column names for quality-related information.

    Returns
    -------
    pd.DataFrame
        Updated `stockpiles_df` with quality parameters extracted as individual columns.
    """
    quality_cols = stockpiles_df.columns[
        stockpiles_df.columns.str.startswith(quality_prefix)
    ]
    quality_ini_dict = {}

    for column in quality_cols:
        for idx, row in stockpiles_df.iterrows():
            parameter = row[column]["parameter"]
            value = row[column]["value"]
            quality_ini_dict.setdefault(parameter, []).append(value)

    # Add the extracted quality values as new columns and drop the original quality columns
    stockpiles_df = pd.concat(
        [stockpiles_df, pd.DataFrame(quality_ini_dict, index=stockpiles_df.index)],
        axis=1,
    ).drop(columns=quality_cols, errors="ignore")

    return stockpiles_df


def process_stockpiles_and_engines(
    stockpiles_df: pd.DataFrame, engines_df: pd.DataFrame
) -> pd.DataFrame:
    """
    Process stockpiles and engines.

    Engines are assigned to stockpiles and initial quality values are extracted.

    Parameters
    ----------
    stockpiles_df : pd.DataFrame
        A `pandas.DataFrame` containing stockpile information.
    engines_df : pd.DataFrame
        A `pandas.DataFrame` containing engine information.

    Returns
    -------
    pd.DataFrame
        Processed `stockpiles_df` with engines assigned and quality values extracted.
    """
    stockpiles_df = assign_engines_to_stockpiles(stockpiles_df, engines_df)
    stockpiles_df = extract_quality_ini_values(stockpiles_df)
    return stockpiles_df


def travel_time(grp: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the travel time between consecutive events within a group.

    This function calculates the time between the end of one event and the start of the next event
    within a grouped DataFrame. If the group contains only one event, the travel time is set to 0.

    Parameters
    ----------
    grp : pd.DataFrame
        A `pandas.DataFrame` containing at least 'start_time' and 'end_time'
        columns. The DataFrame is expected to be pre-grouped by a relevant key
        before being passed to this function.

    Returns
    -------
    pd.DataFrame
        A `pandas.DataFrame` with a single column 'travel_time',
        containing the calculated travel times between consecutive events.
        The index of the returned DataFrame matches the input DataFrame.
    """
    if len(grp) == 1:
        return pd.DataFrame(
            {"travel_time": [grp["start_time"].values[0]]}, index=grp.index
        )
    grp = grp.sort_values(["end_time"])
    end_time = None
    res = []
    for _, row in grp.iterrows():
        _end_time = row["end_time"]
        if end_time is None:
            res.append(row["start_time"])
        else:
            res.append(row["start_time"] - end_time)
        end_time = _end_time
    return pd.DataFrame({"travel_time": res}, index=grp.index)

def json_input_output_to_excel(
    json_input_path: str | Path,
    json_output_path: str | Path,
):
    # Load JSON files
    # Input file
    with open(json_input_path) as fh:
        instance_data = json.load(fh)

    # Output file
    with open(json_output_path) as fh:
        output_data = json.load(fh)

    info_df = pd.DataFrame(
        [instance_data["info"]], columns=["Instance_Name", "Capacity", "Yard"]
    )
    engines_df = pd.DataFrame(instance_data["engines"])
    stockpiles_df = pd.DataFrame(instance_data["stockpiles"])
    stockpiles_quality_df = pd.json_normalize(
        stockpiles_df.pop("qualityIni"), sep="_"
    ).add_prefix("qualityIni_")
    stockpiles_df = pd.concat([stockpiles_df, stockpiles_quality_df], axis=1)
    stockpiles_df = process_stockpiles_and_engines(stockpiles_df, engines_df)

    inputs_df = pd.DataFrame(instance_data["inputs"])
    inputs_quality_df = pd.json_normalize(inputs_df.pop("quality"), sep="_").add_prefix(
        "quality_"
    )
    inputs_df = pd.concat([inputs_df, inputs_quality_df], axis=1)

    # Convert instance_1.json to DataFrames
    outputs_df = pd.DataFrame(instance_data["outputs"])
    outputs_quality_df = pd.json_normalize(
        outputs_df.pop("quality"), sep="_"
    ).add_prefix("quality_")
    outputs_df = pd.concat([outputs_df, outputs_quality_df], axis=1)

    # Explode the outputs_df
    outputs_df = explode_quality_rows(outputs_df, quality_col_prefix="quality_").drop(
        columns=["time"], errors="ignore"
    )
    distances_travel_df = pd.DataFrame(instance_data["distancesTravel"])
    time_travel_df = pd.DataFrame(instance_data["timeTravel"])

    time_travel_df.columns += 1
    time_travel_df.index += 1

    distances_travel_df.columns += 1
    distances_travel_df.index += 1

    from_to_list = []
    for col in time_travel_df.columns:
        for idx in time_travel_df.index:
            from_to_list.append([f"{col} -> {idx}", time_travel_df.loc[idx, col]])
    from_to_df = pd.DataFrame(from_to_list, columns=["from_to", "duration"])

    engines_df[from_to_df["from_to"].to_list()] = -1
    engines_df[from_to_df["from_to"].to_list()] = engines_df[
        from_to_df["from_to"].to_list()
    ].astype(float)

    for idx, row in engines_df.iterrows():
        yards = row["yards"]
        rail = row["rail"]
        stockpiles = []
        for _, stockpile_row in stockpiles_df.iterrows():
            if stockpile_row["yard"] in yards and rail in stockpile_row["rails"]:
                stockpiles.append(stockpile_row["id"])
        for start_stockpile in stockpiles:
            for end_stockpile in stockpiles:
                column_name = f"{start_stockpile} -> {end_stockpile}"
                duration = from_to_df.loc[
                    from_to_df["from_to"] == column_name, "duration"
                ].values[0]
                engines_df.loc[engines_df.index == idx, column_name] = duration

    engines_df[from_to_df["from_to"].to_list()] = engines_df[
        from_to_df["from_to"].to_list()
    ].replace(-1, "")

    objective_df = pd.DataFrame(
        [{"Objective": output_data["objective"], "Gap": output_data["gap"][0]}]
    )
    stacks_df = pd.DataFrame(output_data["stacks"])
    reclaims_df = pd.DataFrame(output_data["reclaims"])

    outputs_df_out = pd.DataFrame(output_data["outputs"])
    outputs_quality_df_out = (
        pd.json_normalize(outputs_df_out.pop("quality"), sep="_")
        .add_prefix("quality_")
    )

    outputs_df_out = pd.concat([outputs_df_out, outputs_quality_df_out], axis=1)
    outputs_df_out = explode_quality_rows(outputs_df_out, quality_col_prefix="quality_")

    if not stacks_df.empty:
        stacks_df["end_time"] = stacks_df["start_time"] + stacks_df["duration"]
        stacks_df["operation"] = "stack"

    if not reclaims_df.empty:
        reclaims_df["end_time"] = reclaims_df["start_time"] + reclaims_df["duration"]
        reclaims_df["operation"] = "reclaim"

    operations_df = (
        pd.concat([xdf for xdf in [stacks_df, reclaims_df] if not xdf.empty])
        .astype({"weight": int})
        .sort_values(["engine", "start_time"])
        .assign(
            travel_time=lambda xdf: (
                xdf.groupby("engine", as_index=False).apply(travel_time)
            )["travel_time"].reset_index(level=0, drop=True)
        )
    )

    stockpiles_final_df = (
        stockpiles_df.merge(
            operations_df.rename(columns={"weight": "weightFinal"})
            .groupby("stockpile")["weightFinal"]
            .sum(),
            left_on="id",
            right_index=True,
            how="left",
        )
        .fillna({"weightFinal": 0})
        .astype({"weightFinal": int})
        .assign(weightFinal=lambda xdf: xdf["weightIni"] - xdf["weightFinal"])
    )

    quality_cols = stockpiles_df.columns.intersection(
        ["Fe", "SiO2", "Al2O3", "P", "+31.5", "-6.3"]
    ).to_list()

    operations_df = operations_df.merge(
        stockpiles_df.rename(columns={"id": "stockpile"})[
            ["stockpile", "weightIni", *quality_cols]
        ],
        on="stockpile",
        how="left",
    ).assign(weightFinal=lambda xdf: xdf["weightIni"] - xdf["weight"])

    final_output_row = [
        operations_df["weight"].sum(),
        "output",
        operations_df["engine"].unique().tolist(),
        operations_df["start_time"].min(),
        operations_df["end_time"].max(),
        1,
        operations_df["end_time"].max(),
        "output_stack",
        operations_df["travel_time"].sum(),
        operations_df["weightFinal"].sum(),
    ]

    for quality_col in quality_cols:
        final_output_row.append(
            (
                (
                    operations_df["weight"]
                    * operations_df[quality_col]
                    / operations_df["weight"].sum()
                ).sum()
            )
        )
    operations_df = pd.concat(
        [
            operations_df,
            pd.DataFrame(
                {
                    col: [value]
                    for col, value in zip(operations_df.columns, final_output_row)
                }
            ),
        ],
        axis=0,
    )

    required_weight = operations_df.loc[
        operations_df["stockpile"] == "output", "weight"
    ].values[0]
    infos_gerais = pd.DataFrame(
        {"Variável": ["Peso Carregamento"], "Valor": [required_weight]}
    )

    engines_yards = (
        stockpiles_final_df[["yard", "engines"]]
        .astype({"engines": str})
        .drop_duplicates()
        .assign(
            engines=lambda xdf: xdf["engines"]
            .str.replace("[", "")
            .str.replace("]", "")
            .str.replace(" ", "")
            .str.replace(",", "")
            .apply(list)
        )
    )
    all_engines = list(
        sorted(
            set([engine for engines in engines_yards["engines"] for engine in engines])
        )
    )
    for engine in all_engines:
        engines_yards[f"Veículo {engine}"] = engines_yards["engines"].apply(
            lambda value: "x" if engine in value else ""
        )

    engines_yards = engines_yards.drop(columns=["engines"]).rename({"yard": "Área"})

    rename_dict = {
        "id": "ID",
        "yard": "Área",
        "weightIni": "Quantidade (ton)",
    }
    final_cols = [*list(rename_dict.values()), *quality_cols]
    stockpiles_final_df = stockpiles_final_df.rename(columns=rename_dict)[final_cols]

    load_rates = (
        engines_df[["id", "speedReclaim"]]
        .astype({"speedReclaim": int})
        .rename(columns={"id": "Veículo", "speedReclaim": "Taxa (ton/min)"})
    )

    from_to_cols = [col for col in engines_df.columns if "->" in col]
    travel_times_dict = {
        "De": [],
        "Para": [],
    }
    for from_to in from_to_cols:
        from_stockpile, to_stockpile = from_to.split(" -> ")
        travel_times_dict["De"].append(from_stockpile)
        travel_times_dict["Para"].append(to_stockpile)
        for engine in all_engines:
            vehicle_travel_times = travel_times_dict.get(f"Veículo {engine}", [])
            engine_row = engines_df.loc[engines_df["id"] == int(engine)]
            time_travel = engine_row[from_to].values[0]
            vehicle_travel_times.append(time_travel)
            travel_times_dict[f"Veículo {engine}"] = vehicle_travel_times

    travel_times_df = pd.DataFrame(travel_times_dict).astype({"De": int, "Para": int})

    rename_dict = {
        "engine": "Veículo",
        "stockpile": "Pilha",
        "weightIni": "Peso Inicial Pilha",
        "weightFinal": "Peso Final Pilha",
        "weight": "Carregamento (ton)",
        "start_time": "Início",
        "end_time": "Fim",
        "duration": "Tempo Carregamento",
        "travel_time": "Tempo Deslocamento",
    }
    operations_df = operations_df.rename(columns=rename_dict)[
        [*list(rename_dict.values()), *quality_cols]
    ]
    operations_df[quality_cols] = operations_df[quality_cols].round(2)
    operations_df = operations_df.fillna("")

    outputs_df_out["check"] = np.where(
        (outputs_df_out["value"] >= outputs_df_out["minimum"])
        & (outputs_df_out["value"] <= outputs_df_out["maximum"]),
        True,
        False,
    )
    rename_dict = {
        "parameter": "Elemento",
        "value": "Valor",
        "minimum": "Mínimo",
        "maximum": "Máximo",
        "goal": "Meta",
        "check": "Check",
    }
    outputs_df_out = outputs_df_out.rename(columns=rename_dict)[
        list(rename_dict.values())
    ]
    return operations_df, outputs_df_out

In [9]:
instance_json_path = "../tests/instance_interactive.json"
output_json_path = "../out/json/out_interactive.json"

operations_df, outputs_df_out = json_input_output_to_excel(
    instance_json_path,
    output_json_path,
)

C:\Users\YG287PG\AppData\Local\Temp\ipykernel_31308\1889619921.py:280: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  engines_df[from_to_df["from_to"].to_list()] = -1
C:\Users\YG287PG\AppData\Local\Temp\ipykernel_31308\1889619921.py:280: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  engines_df[from_to_df["from_to"].to_list()] = -1
C:\Users\YG287PG\AppData\Local\Temp\ipykernel_31308\1889619921.py:280: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poo

In [10]:
operations_df

,Veículo,Pilha,Peso Inicial Pilha,Peso Final Pilha,Carregamento (ton),Início,Fim,Tempo Carregamento,Tempo Deslocamento,Fe,SiO2,Al2O3,P
0,1,6,75000,0.0,75000,0.60,19.83,19.23,0.60,65.40,4.50,4.9,0.06
1,1,17,62500,0.0,62500,21.53,37.56,16.03,1.70,68.10,3.60,4.8,0.06
2,1,18,65000,0.0,65000,39.36,56.03,16.67,1.80,67.00,3.90,4.9,0.07
3,1,20,70000,0.0,70000,58.03,75.98,17.95,2.00,65.90,4.40,5.2,0.09
4,2,8,45000,0.0,45000,1.00,18.31,17.31,1.00,60.30,5.50,4.7,0.06
5,2,9,50000,0.0,50000,18.81,38.04,19.23,0.50,68.30,3.20,4.8,0.07
6,2,7,40000,0.0,40000,38.94,54.32,15.38,0.90,75.00,4.30,5.0,0.08
7,2,1,60000,43750.0,16250,55.53,61.78,6.25,1.21,10.30,2.30,4.5,0.05
8,2,19,67500,36250.0,31250,62.88,74.90,12.02,1.10,69.40,4.00,5.1,0.08
9,2,2,45000,0.0,45000,74.96,92.27,17.31,0.06,76.30,5.60,4.8,0.05


In [14]:
workbook_path = str(Path("../out_interactive.xlsm").resolve())
wb = xw.Book(workbook_path)
sheet = wb.sheets["Resultados"]

In [15]:
last_used_column = sheet.used_range.last_cell.column
last_used_column

13

In [74]:
from typing import Iterable
import matplotlib.pyplot as plt


def is_integer(value):
    if isinstance(value, int):
        return True
    if isinstance(value, str):
        if all(c.isnumeric() for c in value) or re.match(r"\d+\.0+[^1-9]$", value):
            return True
        return False
    
    if isinstance(value, float):
        return int(value) == value

    if isinstance(value, Iterable):
        return all(is_integer(v) for v in value)
    return False


def autofit_columns_from_sheet(sheet, min_width=10):
    """
    Autofit all columns in an Excel sheet.

    Parameters
    ----------
    sheet : xlwings.Sheet
        The sheet where columns need to be autofit.
    min_width : int, default=10
        The minimum width for any column.
    """
    # Get the last used column in the sheet
    last_used_column = sheet.used_range.last_cell.column

    # Loop through each column from the first to the last used column
    for col in range(1, last_used_column + 1):
        column_range = sheet.range(f"{xw.utils.col_name(col)}:{xw.utils.col_name(col)}")
        column_range.autofit()

        # Check if the autofit width is less than the minimum width
        if column_range.column_width < min_width:
            column_range.column_width = min_width


def format_excel_sheet(sheet: xw.Sheet):
    """
    Format all used cells in an Excel sheet.
    
    Function sets specific styles for header, rows with alternating colors,
    and a distinct style for the last row.

    Parameters
    ----------
    sheet : xlwings.Sheet
        The sheet to be formatted.
    """
    sheet.used_range.clear_formats()

    # Get last used row and column
    first_column = sheet.used_range.columns[0].column
    first_row = sheet.used_range.columns[0].row
    last_column = sheet.used_range.last_cell.column
    last_row = sheet.used_range.last_cell.row
    first_column_name = xw.utils.col_name(first_column)
    last_column_name = xw.utils.col_name(last_column)

    # Formatting for the header row
    header_range = sheet.range(f"{first_column_name}{first_row}", f"{last_column_name}{first_row}")

    header_range.color = "#5B80B8"  # Fill color
    header_range.api.Font.Color = 0xFFFFFF  # Font color (White)
    header_range.api.Font.Bold = True
    header_range.api.Font.Size = 11
    header_range.api.Font.Name = "Calibri"
    header_range.api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
    header_range.api.Borders(xw.constants.BordersIndex.xlEdgeBottom).LineStyle = xw.constants.LineStyle.xlContinuous
    header_range.api.Borders(xw.constants.BordersIndex.xlEdgeBottom).Color = 0xFFFFFF  # White
    header_range.api.Borders(xw.constants.BordersIndex.xlEdgeBottom).Weight = xw.constants.BorderWeight.xlThick

    # Formatting for all row cells
    for row in range(2, last_row):  # Start from 2 to avoid header row
        row_range = sheet.range(f"{first_column_name}{row}", f"{last_column_name}{row}")
        row_range.api.Font.Size = 11
        row_range.api.Font.Name = "Calibri"
        row_range.api.Font.Color = 0x000000  # Black
        row_range.color = "#B9C8DE" if row % 2 == 0 else "#DEE6F0"
        row_range.api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
        row_range.api.Borders(xw.constants.BordersIndex.xlEdgeBottom).LineStyle = xw.constants.LineStyle.xlContinuous
        row_range.api.Borders(xw.constants.BordersIndex.xlEdgeBottom).Color = 0xFFFFFF  # White
        row_range.api.Borders(xw.constants.BordersIndex.xlEdgeBottom).Weight = xw.constants.BorderWeight.xlThin

    # Formatting for the last row
    last_row_range = sheet.range(f"{first_column_name}{last_row}", f"{last_column_name}{last_row}")
    last_row_range.color = "#4F81BD"
    last_row_range.api.Font.Color = 0xFFFFFF  # White
    last_row_range.api.Font.Bold = True
    last_row_range.api.Font.Size = 11
    last_row_range.api.Font.Name = "Calibri"
    last_row_range.api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
    last_row_range.api.Borders(xw.constants.BordersIndex.xlEdgeTop).LineStyle = xw.constants.LineStyle.xlDouble
    last_row_range.api.Borders(xw.constants.BordersIndex.xlEdgeTop).Color = 0xFFFFFF  # White
    last_row_range.api.Borders(xw.constants.BordersIndex.xlEdgeTop).Weight = xw.constants.BorderWeight.xlThick


def format_integer_columns(sheet: xw.Sheet):
    """
    Format columns in an Excel sheet where all values are integers.

    Function sets the number format to include a thousand separator
    and zero decimal places.

    Parameters
    ----------
    sheet : xlwings.Sheet
        The sheet to be formatted.
    """
    # Get the last used column and row in the sheet
    last_column = sheet.used_range.last_cell.column
    last_row = sheet.used_range.last_cell.row

    # Iterate through each column in the used range
    for col_index in range(1, last_column + 1):
        column_range = sheet.range((2, col_index), (last_row, col_index))
        values = column_range.value

        # Check if all values in the column are integers
        if all(is_integer(val) or val is None or val == "" for val in values):
            # Apply a number format with thousands' separator and zero decimal places
            column_range.number_format = "#,##0"


def generate_gantt_chart(operations_df, sheet):    
    operations_df = operations_df.iloc[:-1]
    operations_df["Início"] = pd.to_timedelta(operations_df["Início"], unit="m")
    operations_df["Fim"] = pd.to_timedelta(operations_df["Fim"], unit="m")
    operations_df["Tempo Deslocamento (min)"] = (
        pd.to_timedelta(operations_df["Tempo Deslocamento"], unit="m").dt.total_seconds()
        / 60
    )
    
    # Convert the timedeltas to a more plot-friendly format by using hours as float
    operations_df["Início (min)"] = operations_df["Início"].dt.total_seconds() / 60
    operations_df["Fim (min)"] = operations_df["Fim"].dt.total_seconds() / 60
    operations_df["Início (min)"] -= operations_df["Tempo Deslocamento (min)"]
    
    # Create the Gantt plot with additional annotations
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot each operation and add text annotations
    for idx, row in operations_df.iterrows():
        ax.barh(
            row["Veículo"],
            row["Fim (min)"] - row["Início (min)"],
            left=row["Início (min)"],
        )
        # Calculate the position for the text
        mid_point = (row["Início (min)"] + row["Fim (min)"]) / 2
        label = f'SP: {row["Pilha"]}'
        ax.text(
            mid_point,
            row["Veículo"],
            label,
            ha="center",
            va="center",
            color="white",
            fontsize=10,
            usetex=False,
        )
    
    # Formatting the plot
    ax.set_yticks([1, 2])
    ax.set_ylim(0.5, None)
    ax.set_xlabel("Tempo (minutos)")
    ax.set_ylabel("Veículo", fontsize=16)
    ax.set_title("Operações")

    sheet.pictures.add(
        fig,
        name='Gantt',
        update=True,
        anchor=sheet.range((sheet.used_range.last_cell.row + 2, 1)),
    )

In [77]:
class ExcelDataExtractor:
    """Class for extracting data from Excel sheets using xlwings.

    Methods
    -------
    extract_dataframe
        Extracts a DataFrame from a specified range in the sheet.
    """

    def __init__(self, workbook_path: str, sheet_name: str):
        """Initialize the ExcelDataExtractor with workbook path and sheet name.

        Parameters
        ----------
        workbook_path : str
            The path to the Excel workbook.
        sheet_name : str
            The name of the sheet from which data is extracted.
        """
        self.wb = xw.Book(workbook_path)
        self.sheet = self.wb.sheets[sheet_name]

    def extract_dataframe(self, range: str, expand: bool = True) -> pd.DataFrame:
        """Extracts a DataFrame from a specified range in the sheet.

        Parameters
        ----------
        range : str
            The cell range to start extracting data from.
        expand : bool, default=True
            Whether to expand the range to a table.

        Returns
        -------
        pd.DataFrame
            Extracted data as a pandas DataFrame.
        """
        return (
            self.sheet[range]
            .options(pd.DataFrame, expand="table" if expand else None)
            .value.reset_index()
        )


def generate_gantt_chart(operations_df, sheet):
    operations_df = operations_df.iloc[:-1]
    operations_df["Início"] = pd.to_timedelta(operations_df["Início"], unit="m")
    operations_df["Fim"] = pd.to_timedelta(operations_df["Fim"], unit="m")
    operations_df["Tempo Deslocamento (min)"] = (
            pd.to_timedelta(operations_df["Tempo Deslocamento"], unit="m").dt.total_seconds()
            / 60
    )

    # Convert the timedeltas to a more plot-friendly format by using hours as float
    operations_df["Início (min)"] = operations_df["Início"].dt.total_seconds() / 60
    operations_df["Fim (min)"] = operations_df["Fim"].dt.total_seconds() / 60
    operations_df["Início (min)"] -= operations_df["Tempo Deslocamento (min)"]

    # Create the Gantt plot with additional annotations
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot each operation and add text annotations
    for idx, row in operations_df.iterrows():
        ax.barh(
            row["Veículo"],
            row["Fim (min)"] - row["Início (min)"],
            left=row["Início (min)"],
        )
        # Calculate the position for the text
        mid_point = (row["Início (min)"] + row["Fim (min)"]) / 2
        label = f'SP: {row["Pilha"]}'
        ax.text(
            mid_point,
            row["Veículo"],
            label,
            ha="center",
            va="center",
            color="white",
            fontsize=10,
            usetex=False,
        )

    # Formatting the plot
    ax.set_yticks([1, 2])
    ax.set_ylim(0.5, None)
    ax.set_xlabel("Tempo (minutos)")
    ax.set_ylabel("Veículo", fontsize=16)
    ax.set_title("Operações")

    sheet.pictures.add(
        fig,
        name='Gantt',
        update=True,
        anchor=sheet.range((sheet.used_range.last_cell.row + 2, 1)),
    )



In [80]:
extractor = ExcelDataExtractor(workbook_path, "Resultados")
operations_df = extractor.extract_dataframe("A1")
sheet = extractor.sheet

generate_gantt_chart(operations_df, sheet)

C:\Users\YG287PG\AppData\Local\Temp\ipykernel_31308\805608514.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  operations_df["Início"] = pd.to_timedelta(operations_df["Início"], unit="m")
C:\Users\YG287PG\AppData\Local\Temp\ipykernel_31308\805608514.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  operations_df["Fim"] = pd.to_timedelta(operations_df["Fim"], unit="m")
C:\Users\YG287PG\AppData\Local\Temp\ipykernel_31308\805608514.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [81]:
operations_df

,Pilha,Veículo,Peso Inicial Pilha,Peso Final Pilha,Carregamento (ton),Início,Fim,Tempo Carregamento,Tempo Deslocamento,Fe,SiO2,Al2O3,P
0,6.0,1.00,75000.0,0.0,75000.0,1.20,20.43,19.23,1.20,65.40,4.50,4.9,0.06
1,2.0,1.00,45000.0,0.0,45000.0,21.63,33.17,11.54,1.20,76.30,5.60,4.8,0.05
2,11.0,1.00,45000.0,0.0,45000.0,34.17,45.71,11.54,1.00,65.20,4.10,4.9,0.05
3,17.0,1.00,62500.0,0.0,62500.0,45.79,61.82,16.03,0.08,68.10,3.60,4.8,0.06
4,20.0,1.00,70000.0,0.0,70000.0,61.88,79.83,17.95,0.06,65.90,4.40,5.2,0.09
5,18.0,2.00,65000.0,0.0,65000.0,0.80,25.80,25.00,0.80,67.00,3.90,4.9,0.07
6,1.0,2.00,60000.0,28750.0,31250.0,26.50,38.52,12.02,0.70,10.30,2.30,4.5,0.05
7,5.0,2.00,60000.0,43750.0,16250.0,38.68,44.93,6.25,0.16,70.50,4.00,5.1,0.07
8,9.0,2.00,50000.0,0.0,50000.0,45.09,64.32,19.23,0.16,68.30,3.20,4.8,0.07
9,7.0,2.00,40000.0,0.0,40000.0,64.38,79.76,15.38,0.06,75.00,4.30,5.0,0.08


In [21]:
autofit_columns_from_sheet(sheet)

In [39]:
format_excel_sheet(sheet)

In [63]:
format_integer_columns(sheet)

In [ ]:
sheet.used_range.last_cell.column

In [33]:
xw.utils.col_name(sheet.used_range.last_cell.column)

'M'

In [65]:
operations_df = (
    sheet["A1"]
    .options(pd.DataFrame, expand="table" if True else None)
    .value.reset_index()
)

,Pilha,Veículo,Peso Inicial Pilha,Peso Final Pilha,Carregamento (ton),Início,Fim,Tempo Carregamento,Tempo Deslocamento,Fe,SiO2,Al2O3,P
0,6.0,1.00,75000.0,0.0,75000.0,1.20,20.43,19.23,1.20,65.40,4.5,4.9,0.06
1,17.0,1.00,62500.0,0.0,62500.0,20.51,36.54,16.03,0.08,68.10,3.6,4.8,0.06
2,18.0,1.00,65000.0,0.0,65000.0,37.34,54.01,16.67,0.80,67.00,3.9,4.9,0.07
3,9.0,1.00,50000.0,0.0,50000.0,55.81,68.63,12.82,1.80,68.30,3.2,4.8,0.07
4,20.0,1.00,70000.0,0.0,70000.0,68.69,86.64,17.95,0.06,65.90,4.4,5.2,0.09
5,11.0,2.00,45000.0,0.0,45000.0,1.10,18.41,17.31,1.10,65.20,4.1,4.9,0.05
6,1.0,2.00,60000.0,12084.0,47916.0,18.91,37.34,18.43,0.50,10.30,2.3,4.5,0.05
7,7.0,2.00,40000.0,0.0,40000.0,37.50,52.88,15.38,0.16,75.00,4.3,5.0,0.08
8,2.0,2.00,45000.0,0.0,45000.0,53.58,70.89,17.31,0.70,76.30,5.6,4.8,0.05
9,5.0,2.00,60000.0,10417.0,49583.0,70.95,90.02,19.07,0.06,70.50,4.0,5.1,0.07


In [76]:
generate_gantt_chart(operations_df, sheet)

C:\Users\YG287PG\AppData\Local\Temp\ipykernel_31308\4166933204.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  operations_df["Início"] = pd.to_timedelta(operations_df["Início"], unit="m")
C:\Users\YG287PG\AppData\Local\Temp\ipykernel_31308\4166933204.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  operations_df["Fim"] = pd.to_timedelta(operations_df["Fim"], unit="m")
C:\Users\YG287PG\AppData\Local\Temp\ipykernel_31308\4166933204.py:136: SettingWithCopyWarning: 
A value is trying to be set on a c

In [56]:
x = ""

re.match(r"\d+\.0+[^1-9]$", x)

In [61]:
x = float(10.50)
y = 10
x, y
int(x) == x

False

In [72]:
?sheet.pictures.add

In [73]:
anchor = sheet.range((sheet.used_range.last_cell.row + 2, 1))
# sheet.used_range.last_cell.row

<Range [out_interactive.xlsm]Resultados!$A$14>